# Loading the data into the DB.

Following the concepts on the different [means](/HowToGuides/start/Ingestion) of ingesting the data,
we will use the Cookbook dataset to have a working example with each of the type defined.

We are going to use each of the documented methods to ingest the same dataset.

## Data Models

A popular way to define the schema in python is using pydantic, and we shall use the same to create the associations of our Cookbook.


In [67]:
from typing import List
from aperturedb.DataModels import  ImageDataModel, IdentityDataModel

class Ingredient(IdentityDataModel):
    ingredient_name: str

class Dish(ImageDataModel):
    contributor: str
    dish_name: str
    location: str
    food_tags: str
    caption: str
    recipe_url: str
    dish_id: int
    ingredients: List[Ingredient]


### Create objects of these classes.

We will provision the data using a json file, prepared from [script](https://github.com/aperture-data/Cookbook/blob/main/scripts/create_nested_json.py)

Example line from the json file:

:::note title="Sample from the dishes.json"
```
{
        "url": "https://github.com/aperture-data/Cookbook/blob/0e01a8dca2a995e311508ddbf10a9de13c67bb9b/images/001%20Large.jpeg",
        "contributor": "gautam",
        "dish_name": "rajma chawal",
        "location": "NJ",
        "food_tags": "Indian",
        "caption": "Beans with rice",
        "constraint_id": 1,
        "dish_id": 1,
        "recipe_url": "https://www.tarladalal.com/rajma-chawal-punjabi-rajma-chawal-4951r",
        "ingredients": [
            {
                "ingredient_name": "rajma"
            },
            {
                "ingredient_name": "rice"
            }
        ]
    }
```
:::

These objects can be passed to a function called generate_add_query which takes care of generating the queries that ApertureDB can execute to persist the objects on the DB.

In [ ]:
from aperturedb.Query import generate_add_query
from aperturedb.CommonLibrary import  execute_query, create_connector
import json


with open("dishes.json") as ins:
    client = create_connector()
    dishes = json.load(ins)
    for dish in dishes:

        dish = Dish(**dish)

        query, blobs, _ = generate_add_query(dish)
        result, response, output_blobs = execute_query(client, query, blobs)

        if result != 0:
            print(response)
            break



## Ingesting using the CSV Parsers.

Let's ingest the same data. We shall pre process the csvs to confirm to aperuredb's input CSV format.

In [ ]:
from aperturedb.ImageDataCSV import ImageDataCSV
from aperturedb.CommonLibrary import create_connector, execute_query

dishes = ImageDataCSV("../../images")
client = create_connector()

for query, blobs in dishes:
    result, response, output_blobs = execute_query(client, query, blobs)

    if result != 0:
        print(response)
        break